<div id='loot_liquidity_outline'> </div>


* # outline
    * imports
    * setup web3 module
    * ```LootLiquidity``` class definition
        * getting price
        * compare to current pool,
            * if more than  

[back to top](#loot_liquidity_outline) 

In [ ]:
from web3 import Web3, EthereumTesterProvider


In [ ]:
import json
from web3 import Web3


infura_url = f"https://eth.llamarpc.com"
web3 = Web3(Web3.HTTPProvider(infura_url))

In [ ]:
import requests
import json

def get_contract_abi(etherscan_api_key, contract_address):
    url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={etherscan_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data["status"] == "1":
            return json.loads(data["result"])
        else:
            raise Exception("Error fetching ABI: " + data["message"])
    else:
        raise Exception("Error fetching ABI: HTTP " + str(response.status_code))

# Example usage
etherscan_api_key = "YOUR_ETHERSCAN_API_KEY"  # Replace with your Etherscan API key
contract_address = "YOUR_CONTRACT_ADDRESS"   # Replace with the contract address
try:
    abi = get_contract_abi(etherscan_api_key, contract_address)
    print("ABI fetched successfully.")
except Exception as e:
    print(str(e))


In [ ]:
from web3 import Web3

class LootLiquidity:
    def __init__(self, rpc_url):
        self.rpc_url = rpc_url
        self.web3 = Web3(Web3.HTTPProvider(self.rpc_url))
        if self.web3.is_connected():
            print("Connected to Ethereum blockchain.")
        else:
            print("Failed to connect to Ethereum blockchain.")
            raise Exception("Unable to connect to Ethereum blockchain.")

    def fetch_current_pool_price_range(self, pool_address):
        # Load the ABI for the Uniswap v3 pool contract
        uniswap_v3_pool_abi = json.loads("YOUR_ABI_HERE")  # Replace with actual ABI

        # Create a contract object using the ABI and the pool's address
        pool_contract = self.web3.eth.contract(address=pool_address, abi=uniswap_v3_pool_abi)

        try:
            # Fetching slot0 which contains the current price and other data
            slot0 = pool_contract.functions.slot0().call()
            current_price = slot0[0]

            # Convert current price to a more readable format if necessary
            # This depends on how prices are represented in the contract

            # Return the current price (this can be expanded to include more data)
            return {"current_price": current_price}

        except Exception as e:
            print(f"An error occurred: {e}")
            return None
        

# Example usage
rpc_url = f"https://eth.llamarpc.com"
loot_liquidity = LootLiquidity(rpc_url)

# Fetching the pool price range for a specific pool
pool_address = "0xc91b7b39bbb2c733f0e7459348fd0c80259c8471"  # Replace with the actual pool contract address
price_range = loot_liquidity.fetch_current_pool_price_range(pool_address)
print(f"Price Range: {price_range}")






# GNS / ETH  %0.3 ---> https://info.uniswap.org/#/arbitrum/pools/0xc91b7b39bbb2c733f0e7459348fd0c80259c8471
# GNS / USDC %0.3 ---> https://info.uniswap.org/#/arbitrum/pools/0xaa9e653252ed9e87a9bd545b974efbfb2f389f3f